### Guidelines for Choosing the Number of Hidden Layers and Neurons in an ANN

* **Start Simple** — Begin with a small network and increase complexity only if performance is insufficient.

* **Increase Gradually** — Add layers or neurons step by step while monitoring performance improvements.

* **Use Grid Search / Random Search** — Experiment with different layer and neuron combinations automatically.

* **Apply Cross-Validation** — Validate model performance across multiple data splits to avoid overfitting.

* **Follow Basic Heuristics**
    * Hidden layer neurons typically lie between input and output layer sizes.
    * Start with 1–2 hidden layers before adding more.

* **Monitor Overfitting** — More layers/neurons can lead to overfitting; use regularization or early stopping if needed.

* **Balance Performance and Complexity** — Prefer simpler models if performance differences are small.

* **Use Early Stopping** — Stop training when validation performance stops improving.

* **Consider Dataset Size** — Larger datasets can support deeper networks; small datasets usually need simpler models.

* **Track Experiments** — Log architectures and results for comparison and future reference.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [4]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [5]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [6]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [7]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/100


c:\Users\Srilathaa Vasu\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Srilathaa Vasu\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7086 - loss: 0.5905
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - accuracy: 0.8016 - loss: 0.4612
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - accuracy: 0.8094 - loss: 0.4353
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8144 - loss: 0.4211
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - accuracy: 0.8213 - loss: 0.4093
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - accuracy: 0.8280 - loss: 0.3981
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8344 - loss: 0.3869  
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step - accuracy: 0.8461 - loss: 0.3761
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - accuracy: 0.8501 - loss: 0.3673
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step - accuracy: 0.8553 - loss: 0.3605
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.8570 - loss: 0.3555
Epoch 12/100
250/250 ━━━━━━━━━━━━━━